# Alphie

In [1]:
# #4 ****
# import os
# from dotenv import find_dotenv, load_dotenv
# from slack_sdk import WebClient
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# from flask import Flask, request
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.chains import LLMChain  # Corrected import
# from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# import re
# from PyPDF2 import PdfReader
# from langchain_openai import OpenAIEmbeddings

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
# SLACK_BOT_USER_ID = os.environ["SLACK_BOT_USER_ID"]

# # Initialize the Slack app and Flask app
# app = App(token=SLACK_BOT_TOKEN)
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # Initialize the OpenAI-based language model with conversational capabilities
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# # Create a memory buffer for maintaining context in conversations
# memory = ConversationBufferMemory()

# # System prompt template to define AI assistant's behavior
# system_template = """
# You are Alphie, a conversational AI assistant at New Mexico Tax & Rev. 
# Your role is to engage in meaningful conversations and help with a variety of tasks, including answering general questions, solving coding issues, and analyzing the Current New Mexico Statutes Annotated 1978.
# Provide thoughtful and friendly responses to all user questions, regardless of topic.
# """

# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# # Human message prompt to integrate user input into the conversation
# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# # Create a conversational prompt with system and human messages
# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Create a conversational chain with memory for maintaining context
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Function to extract text from PDF
# def extract_text_from_pdf(pdf_path):
#     reader = PdfReader(pdf_path)
#     raw_text = ""
#     for page in reader.pages:
#         text = page.extract_text()
#         if text:
#             raw_text += text + "\n"
#     return raw_text

# # Function to clean and process text
# def clean_text(text):
#     # Remove unnecessary whitespace and normalize the text
#     text = re.sub(r'\s+', ' ', text)  # Replacing multiple whitespaces with a single space
#     text = text.strip()  # Strips leading and trailing whitespaces
#     return text

# # Function to handle PDF-based analysis
# def analyze_pdf(pdf_path, question):
#     raw_text = extract_text_from_pdf(pdf_path)
#     cleaned_text = clean_text(raw_text)

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=300,
#         separators=["\n", " ", "."],  # Better context splitting
#     )
#     texts = text_splitter.split_text(cleaned_text)

#     embeddings = OpenAIEmbeddings()
#     docsearch = FAISS.from_texts(texts, embeddings)

#     chain = load_qa_chain(OpenAI(), chain_type="refine")

#     docs = docsearch.similarity_search(question, k=5)  # Broader context
#     answer = chain.run(input_documents=docs, question=question)

#     return answer

# # Slack event handler for message events
# @app.event("message")
# def handle_message_events(event, say):
#     user_text = event.get("text", "").lower()

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Alphie, your friendly AI assistant at New Mexico Tax & Rev. How can I help you today?")
#         return

#     # Detect if it's a coding-related question
#     if "code" in user_text or "coding" in user_text or "programming" in user_text:
#         # Use the conversational chain to handle general/coding questions
#         response = chain.run({"user_input": user_text})  # Provide user input
#         say(response)
#         return

#     # If the message is asking about Article 17 or any specific part of the statutes
#     if "article 17" in user_text:
#         question = user_text.split("article 17")[-1].strip()
#         response = analyze_pdf("Article 17.pdf", question)

#         if not response or "I don't know" in response:
#             say("I'm not sure I understood that. Could you please rephrase your question about Article 17?")
#         else:
#             say(response)
#         return

#     # Default response
#     say("I'm not sure about that. Can I help with something else?")

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # Initialize the Flask app
# if __name__ == "__main__":
#     flask_app.run()


In [2]:
# The Base Version

import os
import re
from flask import Flask, request
from slack_sdk import WebClient
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
import pandas as pd
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import find_dotenv, load_dotenv

# Load environment variables
load_dotenv(find_dotenv())

# Set Slack API credentials
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# Initialize the Slack app and Flask app only once
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)  # Corrected initialization
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# OpenAI-based language model with conversational capabilities
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create a memory buffer for maintaining context in conversations
memory = ConversationBufferMemory()

# Create system and human prompts
system_template = """
You are Alphie, a conversational AI assistant at New Mexico Tax & Rev. 
Your role is to engage in meaningful conversations and help with various tasks.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Conversational chain with memory
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    raw_text = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            raw_text += text + "\n"
    return raw_text

# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace with single space
    text = text.strip()  # Strip leading and trailing whitespace
    return text

# Function to analyze PDF content
def analyze_pdf(pdf_path, question):
    raw_text = extract_text_from_pdf(pdf_path)
    cleaned_text = clean_text(raw_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        separators=["\n", " ", "."],  # Better context splitting
    )
    texts = text_splitter.split_text(cleaned_text)

    embeddings = OpenAIEmbeddings()
    docsearch = FAISS.from_texts(texts, embeddings)

    qa_chain = load_qa_chain(OpenAI(), chain_type="refine")

    docs = docsearch.similarity_search(question, k=5)  # Context for similarity search
    answer = qa_chain.run(input_documents=docs, question=question)

    return answer

# Load the mapping from the CSV file for keyword-to-PDF mapping
csv_path = "C:/Users/asifr\OneDrive - State of New Mexico\Documents/GitHub/langchain-experiments/slack/output_tables/article_titles.csv"  # Update with the correct path
pdf_dir = "articles_output"  # Directory with PDFs

article_pdf_map = {}
df = pd.read_csv(csv_path)

# Populate the dictionary with ARTICLE numbers and short titles
for _, row in df.iterrows():
    article_number = row["ARTICLE Number"]
    short_title = row["Short Title"]

    pdf_name = f"ARTICLE {article_number}.pdf"
    pdf_path = os.path.join(pdf_dir, pdf_name)

    article_pdf_map[article_number] = pdf_path
    article_pdf_map[short_title.lower()] = pdf_path

# Function to get the PDF path based on a keyword
def get_pdf_path_from_keyword(keyword):
    keyword = keyword.lower()
    for key in article_pdf_map:
        if keyword in key:
            return article_pdf_map[key]
    return None

# Slack event handler for message events
@app.event("message")
def handle_message_events(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Alphie, your friendly AI assistant. How can I help you today?")
        return

    # Check if the message contains a keyword that maps to a PDF
    pdf_path = get_pdf_path_from_keyword(user_text)

    if pdf_path:
        question = re.sub(r"(" + "|".join(article_pdf_map.keys()) + ")", "", user_text).strip()
        
        response = analyze_pdf(pdf_path, question)

        if not response or "I don't know" in response:
            say("I'm not sure I understood that. Could you please rephrase your question?")
        else:
            say(response)
        return

    # Default response for other queries
    response = chain.run({"user_input": user_text})
    say(response)

# Flask endpoint for Slack events
@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    return handler.handle(request)

# # Initialize the Flask app
if __name__ == "__main__":
    flask_app.run()


In [3]:
# !pip install slack_bolt slack_sdk Flask yfinance


In [4]:
# import os
# import re
# import yfinance as yf
# from flask import Flask, request
# from slack_sdk import WebClient
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# import pandas as pd
# import fitz  # PyMuPDF
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
# from langchain.chains import LLMChain
# from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from dotenv import find_dotenv, load_dotenv

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]

# # Initialize the Slack app and Flask app
# app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # Initialize the OpenAI-based language model
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# # Create a memory buffer for conversational context
# memory = ConversationBufferMemory()

# # Create system and human prompts
# system_template = """
# You are Alphie, a conversational AI assistant at New Mexico Tax & Rev. 
# Your role is to engage in meaningful conversations and help with various tasks.
# """
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Create a conversational chain with memory
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Function to extract text from PDF
# def extract_text_from_pdf(pdf_path):
#     reader = PdfReader(pdf_path)
#     raw_text = ""
#     for page in reader.pages:
#         text = page.extract_text()
#         if text:
#             raw_text += text + "\n"
#     return raw_text

# # Function to clean text
# def clean_text(text):
#     # Replace multiple whitespace with a single space, then strip leading and trailing whitespace
#     text = re.sub(r'\s+', ' ', text)
#     text = text.strip()
#     return text

# # Function to analyze PDF content
# def analyze_pdf(pdf_path, question):
#     raw_text = extract_text_from_pdf(pdf_path)
#     cleaned_text = clean_text(raw_text)

#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=1000,
#         chunk_overlap=300,
#         separators=["\n", " ", "."],
#     )
#     texts = text_splitter.split_text(cleaned_text)

#     embeddings = OpenAIEmbeddings()
#     docsearch = FAISS.from_texts(texts, embeddings)

#     qa_chain = load_qa_chain(OpenAI(), chain_type="refine")

#     docs = docsearch.similarity_search(question, k=5)
#     answer = qa_chain.run(input_documents=docs, question=question)

#     return answer

# # Load the CSV with keyword-to-PDF mapping
# csv_path = "C:/Users/asifr\OneDrive - State of New Mexico\Documents/GitHub/langchain-experiments/slack/output_tables/article_titles.csv"  # Correct the path to your CSV
# pdf_dir = "articles_output"  # Directory with PDFs

# article_pdf_map = {}
# df = pd.read_csv(csv_path)

# # Populate the dictionary with ARTICLE numbers and short titles
# for _, row in df.iterrows():
#     article_number = row["ARTICLE Number"]
#     short_title = row["Short Title"]

#     pdf_name = f"ARTICLE {article_number}.pdf"
#     pdf_path = os.path.join(pdf_dir, pdf_name)

#     article_pdf_map[article_number] = pdf_path
#     article_pdf_map[short_title.lower()] = pdf_path

# # Function to get the PDF path based on a keyword
# def get_pdf_path_from_keyword(keyword):
#     keyword = keyword.lower()
#     for key in article_pdf_map:
#         if keyword in key:
#             return article_pdf_map[key]
#     return None

# # Slack event handler for message events
# @app.event("message")
# def handle_message_events(event, say):
#     user_text = event.get("text", "").lower()

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Alphie, your AI assistant. How can I help you today?")
#         return

#     # Handle commodity price queries
#     if "oil price" in user_text or "crude oil" in user_text:
#         crude_oil = yf.Ticker("CL=F")
#         data = crude_oil.history(period="1d")
#         latest_date = data.index[-1]
#         latest_price = data["Close"].iloc[-1]

#         say(f"Latest crude oil price on {latest_date}: ${latest_price:.2f}")
#         return

#     if "gold price" in user_text:
#         gold = yf.Ticker("GC=F")
#         data = gold.history(period="1d")
#         latest_date = data.index[-1]
#         latest_price = data["Close"].iloc[-1]

#         say(f"Latest gold price on {latest_date}: ${latest_price:.2f}")
#         return

#     # Check if message contains a keyword that maps to a PDF
#     pdf_path = get_pdf_path_from_keyword(user_text)

#     if pdf_path:
#         question = re.sub(r"(" + "|".join(article_pdf_map.keys()) + ")", "", user_text).strip()
#         response = analyze_pdf(pdf_path, question)

#         if not response or "I don't know" in response:
#             say("I'm not sure I understood that. Could you rephrase your question?")
#         else:
#             say(response)
#         return

#     # Default response for general queries
#     response = chain.run({"user_input": user_text})
#     say(response)

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # Initialize and run the Flask app
# if __name__ == "__main__":
#     flask_app.run()


# Fedrick

In [5]:
# import os
# import re
# from flask import Flask, request
# from slack_sdk import WebClient
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# from dotenv import find_dotenv, load_dotenv
# from fredapi import Fred
# from langchain.llms import OpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from langchain.chains import LLMChain
# from langchain.chat_models import ChatOpenAI

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials and FRED API key
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
# FRED_API_KEY = os.environ["FRED_API_KEY"]

# # Initialize the Slack app and Flask app
# app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # Initialize the OpenAI-based language model
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# # Create a memory buffer for conversational context
# memory = ConversationBufferMemory()

# # Create system and human prompts for Fredrick-specific conversations
# system_template = """
# You are Fredrick, an AI assistant working at the New Mexico Tax & Revenue Department. 
# Your role is to assist with data-related queries, provide economic data on demand, and engage in meaningful conversations.
# """
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Create a conversational chain with memory for FRED-related queries
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Initialize the FRED API
# fred = Fred(api_key=FRED_API_KEY)

# # Function to fetch data from FRED based on a series ID
# def fetch_data_from_fred(series_id):
#     try:
#         series = fred.get_series(series_id)
#         latest_value = series.iloc[-1
# ]
#         latest_date = series.index[-1]
#         return latest_date, latest_value
#     except Exception as e:
#         return None, None

# # Slack event handler for all messages
# @app.event("message")
# def handle_all_messages(event, say):
#     user_text = event.get("text", "").lower()

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Fredrick, your AI assistant at the New Mexico Tax & Revenue Department. How can I help you today?")
#         return

#     # Respond to FRED-related queries
#     if "gdp" in user_text:
#         date, value = fetch_data_from_fred("GDP")
#         if date and value:
#             say(f"Latest GDP data from FRED: {date} - ${value:,.2f} (in billions)")
#         else:
#             say("I couldn't fetch the latest GDP data. Please try again later.")
#         return

#     if "cpi" in user_text:
#         date, value = fetch_data_from_fred("CPIAUCSL")
#         if date and value:
#             say(f"Latest CPI data from FRED: {date} - {value}")
#         else:
#             say("I couldn't fetch the latest CPI data. Please try again later.")
#         return

#     if "unemployment rate" in user_text:
#         date, value = fetch_data_from_fred("UNRATE")
#         if date and value:
#             say(f"Latest Unemployment Rate from FRED: {date} - {value}%")
#         else:
#             say("I couldn't fetch the latest unemployment rate. Please try again later.")
#         return

#     # Handle general queries with conversational chain
#     response = chain.run({"user_input": user_text})
#     say(response)

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # Initialize and run the Flask app
# # # Initialize and run the Flask app
# if __name__ == "__main__":
#     flask_app.run()



In [6]:
# import os
# import re
# from flask import Flask, request
# from slack_sdk import WebClient
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# from dotenv import find_dotenv, load_dotenv
# from fredapi import Fred
# from langchain.llms import OpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from langchain.chains import LLMChain
# from langchain.chat_models import ChatOpenAI
# import pandas as pd

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials and FRED API key
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
# FRED_API_KEY = os.environ["FRED_API_KEY"]

# # Initialize the Slack app and Flask app
# app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # Initialize the OpenAI-based language model
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# # Create a memory buffer for conversational context
# memory = ConversationBufferMemory()

# # Create system and human prompts for Fredrick-specific conversations
# system_template = """
# You are Fredrick, an AI assistant working at the New Mexico Tax & Revenue Department. 
# Your role is to assist with data-related queries, provide economic data on demand, and engage in meaningful conversations.
# """
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Create a conversational chain with memory for FRED-related queries
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Initialize the FRED API
# fred = Fred(api_key=FRED_API_KEY)

# # Function to fetch data from FRED based on a series ID and a date range
# def fetch_historical_data_from_fred(series_id, start_date, end_date):
#     try:
#         series = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
#         if not series.empty:
#             # Convert the series to a simple text-based table
#             table_data = pd.DataFrame(series, columns=["Value"]).reset_index()
#             table_data.columns = ["Date", "Value"]
            
#             # Format as a Slack-friendly markdown table
#             table_str = "```"  # Slack code block
#             table_str += "{:<12} {:>10}\n".format("Date", "Value")
#             for index, row in table_data.iterrows():
#                 table_str += "{:<12} {:>10}\n".format(row["Date"].strftime("%Y-%m-%d"), f"{row['Value']:.2f}")
#             table_str += "```"
#             return table_str
#         else:
#             return "No data found for the given date range."
#     except Exception as e:
#         return f"An error occurred while fetching data: {str(e)}"

# # Slack event handler for all messages
# @app.event("message")
# def handle_all_messages(event, say):
#     user_text = event.get("text", "").lower()

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Fredrick, your AI assistant at the New Mexico Tax & Revenue Department. How can I help you today?")
#         return

#     # Respond to FRED-related queries
#     if "gdp" in user_text:
#         historical_data = fetch_historical_data_from_fred("GDP", "2020-01-01", "2023-01-01")
#         say(f"Historical GDP data from FRED:\n{historical_data}")
#         return

#     if "cpi" in user_text:
#         historical_data = fetch_historical_data_from_fred("CPIAUCSL", "2020-01-01", "2023-01-01")
#         say(f"Historical CPI data from FRED:\n{historical_data}")
#         return

#     if "unemployment rate" in user_text:
#         historical_data = fetch_historical_data_from_fred("UNRATE", "2020-01-01", "2023-01-01")
#         say(f"Historical Unemployment Rate data from FRED:\n{historical_data}")
#         return

#     # Handle general queries with conversational chain
#     response = chain.run({"user_input": user_text})
#     say(response)

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # Initialize and run the Flask app
# if __name__ == "__main__":
#     flask_app.run()


In [7]:
# Base ***

# import os
# from flask import Flask, request
# from slack_bolt import App
# from slack_bolt.adapter.flask import SlackRequestHandler
# from dotenv import find_dotenv, load_dotenv
# from fredapi import Fred
# from langchain.chat_models import ChatOpenAI
# from langchain.memory import ConversationBufferMemory
# from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from langchain.chains import LLMChain
# import pandas as pd
# import re

# # Load environment variables
# load_dotenv(find_dotenv())

# # Set Slack API credentials and FRED API key
# SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
# SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
# FRED_API_KEY = os.environ["FRED_API_KEY"]

# # Initialize the Slack app and Flask app
# app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
# flask_app = Flask(__name__)
# handler = SlackRequestHandler(app)

# # Initialize the OpenAI-based language model
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# # Create a memory buffer for conversational context
# memory = ConversationBufferMemory()

# # Create system and human prompts for Fredrick-specific conversations
# system_template = """
# You are Fredrick, an AI assistant working at the New Mexico Tax & Revenue Department. 
# Your role is to assist with data-related queries, provide economic data on demand, and engage in meaningful conversations.
# """
# system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# human_template = "User says: {user_input}."
# human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# # Create a conversational chain with memory for FRED-related queries
# chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# # Initialize the FRED API
# fred = Fred(api_key=FRED_API_KEY)

# # Function to fetch data from FRED based on a series ID
# def fetch_historical_data_from_fred(series_id):
#     try:
#         # Fetch the entire time series
#         series = fred.get_series(series_id)
#         if series.empty:
#             return "No data found for the given series."

#         # Convert the series to a simple text-based table
#         table_data = pd.DataFrame(series, columns=["Value"]).reset_index()
#         table_data.columns = ["Date", "Value"]

#         # Format as a Slack-friendly markdown table, using raw dates
#         table_str = "```"  # Slack code block
#         table_str += "{:<15} {:>10}\n".format("Date", "Value")

#         for _, row in table_data.iterrows():
#             date = str(row["Date"])  # Convert the date to string without formatting
#             table_str += "{:<15} {:>10}\n".format(date, f"{row['Value']:.2f}")

#         table_str += "```"
#         return table_str

#     except Exception as e:
#         return f"An error occurred while fetching data: {str(e)}"

# # Function to get series ID based on a keyword
# def get_series_id_from_keyword(keyword):
#     # Search for FRED series based on a keyword
#     search_result = fred.search(keyword)
#     if search_result.empty:
#         return None
#     # Get the first result from the search, or return None if empty
#     return search_result.iloc[0]["id"] if not search_result.empty else None

# # Slack event handler for all messages
# @app.event("message")
# def handle_all_messages(event, say):
#     user_text = event.get("text", "").lower()

#     # Respond to greetings
#     if user_text in ["hi", "hello", "hey"]:
#         say("Hello! I'm Fredrick, your AI assistant at the New Mexico Tax & Revenue Department. How can I help you today?")
#         return

#     # Determine which data to fetch based on the user's request
#     keyword = re.search(r"\b(\w+)\b", user_text).group(0)  # Extract the first word as the keyword
#     series_id = get_series_id_from_keyword(keyword)

#     if series_id:
#         historical_data = fetch_historical_data_from_fred(series_id)
#         if "error" not in historical_data.lower():  # Check if there's an error in the response
#             say(f"Data for {keyword.upper()} from FRED:\n{historical_data}")
#         else:
#             say(historical_data)  # If there's an error in fetching data, return the error message
#     else:
#         say("Sorry, I couldn't find any data for that query.")

# # Flask endpoint for Slack events
# @flask_app.route("/slack/events", methods=["POST"])
# def slack_events():
#     return handler.handle(request)

# # Initialize and run the Flask app
# if __name__ == "__main__":
#     flask_app.run()


In [8]:
import os
from flask import Flask, request
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
from dotenv import find_dotenv, load_dotenv
from fredapi import Fred
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
import pandas as pd
import re
from datetime import datetime

# Load environment variables
load_dotenv(find_dotenv())

# Set Slack API credentials and FRED API key
SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_SIGNING_SECRET = os.environ["SLACK_SIGNING_SECRET"]
FRED_API_KEY = os.environ["FRED_API_KEY"]

# Initialize the Slack app and Flask app
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# Initialize the OpenAI-based language model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create a memory buffer for conversational context
memory = ConversationBufferMemory()

# Create system and human prompts for Fredrick-specific conversations
system_template = """
You are Fredrick, an AI assistant working at the New Mexico Tax & Revenue Department. 
Your role is to assist with data-related queries, provide economic data on demand, and engage in meaningful conversations.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create a conversational chain with memory for FRED-related queries
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

# Initialize the FRED API
fred = Fred(api_key=FRED_API_KEY)

# Function to fetch data from FRED based on a series ID
def fetch_historical_data_from_fred(series_id):
    source_url = f"https://fred.stlouisfed.org/series/{series_id}"  # Source URL

    try:
        # Fetch the entire time series
        series = fred.get_series(series_id)
        if series.empty:
            return f"No data found for the given series. You can check the source here: {source_url}"

        # Create a dataframe and get the first 5 and last 5 rows
        table_data = pd.DataFrame(series, columns=["Value"]).reset_index()
        table_data.columns = ["Date", "Value"]

        # Get the first 5 and last 5 rows
        first_rows = table_data.head(5)
        last_rows = table_data.tail(5)

        # Format as a Slack-friendly markdown table
        table_str = "```"  # Slack code block
        table_str += "{:<15} {:>10}\n".format("Date", "Value")

        # Add the first 5 rows
        for _, row in first_rows.iterrows():
            date = str(row["Date"])  # Use raw date
            table_str += "{:<15} {:>10}\n".format(date, f"{row['Value']:.2f}")

        # Add ellipses to indicate omitted rows
        table_str += "... [omitted] ...\n"

        # Add the last 5 rows
        for _, row in last_rows.iterrows():
            date = str(row["Date"])  # Use raw date
            table_str += "{:<15} {:>10}\n".format(date, f"{row['Value']:.2f}")

        table_str += "```"

        # Return the data table along with the source link
        return f"{table_str}\nSource: {source_url}"

    except Exception as e:
        # If there's an error, return the error message along with the source URL
        return f"An error occurred while fetching data: {str(e)}. You can check the source here: {source_url}"

# Function to get series ID based on a keyword
def get_series_id_from_keyword(keyword):
    # Search for FRED series based on a keyword
    search_result = fred.search(keyword)
    if search_result.empty:
        return None
    # Get the first result from the search
    return search_result.iloc[0]["id"]

# Slack event handler for all messages
@app.event("message")
def handle_all_messages(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hello! I'm Fredrick, your AI assistant at the New Mexico Tax & Revenue Department. How can I help you today?")
        return

    # Determine which data to fetch based on the user's request
    keyword = re.search(r"\b(\w+)\b", user_text).group(0)  # Extract the first word as the keyword
    series_id = get_series_id_from_keyword(keyword)

    if series_id:
        historical_data = fetch_historical_data_from_fred(series_id)
        say(f"Data for {keyword.upper()} from FRED:\n{historical_data}")  # Provide data with source link
    else:
        # If the series ID is not found, provide a default message with a source URL
        say("Sorry, I couldn't find any data for that query. You can check the FRED website for more information.")

# Flask endpoint for Slack events
@flask_app.route("/slack/events", methods=["POST"])
def slack_events():
    return handler.handle(request)


# Initialize and run the Flask app
if __name__ == "__main__":
    flask_app.run()

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/May/2024 16:10:25] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 16:10:26] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 16:10:31] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 16:10:33] "POST /slack/events HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2024 16:10:46] "POST /slack/events HTTP/1.1" 200 -
Failed to run listener function (error: Out of bounds nanosecond timestamp: 1210-01-01, at position 0)
Traceback (most recent call last):
  File "conversion.pyx", line 326, in pandas._libs.tslibs.conversion._TSObject.ensure_reso
  File "np_datetime.pyx", line 683, in pandas._libs.tslibs.np_date